In [1]:
import argparse
import numpy as np
import os.path
import scipy.linalg as sla
import sys
import datetime
import os
import psutil
import sparse
import operator

from scipy.sparse import coo_matrix

from dask.distributed import Client
from dask.bag import read_text
import dask.array as da

from sklearn.datasets import make_sparse_coded_signal

In [2]:
#Inputs
'''
These inputs were copied from the pyspark version.
I may not need all of these.

'''


input_data = ''
T = 100 #Rows
P = 100 #Cols

#Optional
R = int(0.07 * P) #Enforces sparcity 
M = 15 #Atoms of the learned dictionary
E = 0.01 #Epsilon
norm = True

print(f'Sparcity:\t{R}')
#Spark Options, look for similar Dask elements
partitions = 4*4  #Chunks?
execmem = '8g'    

#Outputs
file_D = 'D.txt'
file_z = 'z.txt'
prefix = False

Sparcity:	7


In [3]:
client = Client()

In [24]:
max_iterations = P*10
seed = np.random.randint(max_iterations +1, high = 4294967295)
S, D, SV = make_sparse_coded_signal(T,M,P,R,seed)
S = da.from_array(S)
S -= S.mean()
S /= sla.norm(S,axis = 0)


u_new = da.zeros(T)
v = da.zeros(P)

In [25]:
for m in range(M):
    #We need to broadcast the seed to create the initial random vector 
    #seed = np.random.randint(max_iterations +1, high = 4294967295)
    _SEED_ = client.scatter(seed, broadcast=True)
    np.random.seed(_SEED_.result())
    
    #Create a dense random vector
    u_old = da.random.random(T)
    u_old -= u_old.mean()
    u_old /= sla.norm(u_old,axis = 0)
    num_iterations = 0
    delta = 2*E
    print('Lets start the inner loop')
    # Start the inner loop: this learns a single atom.
    while num_iterations < max_iterations and delta > E:
        
        _U_ = client.scatter(u_old, broadcast=True) 
        v = da.matmul(_U_.result(),S) #May get an error here because S may be a future instead of a dask array
        print(v.shape)
        #Grab the indices and data of the top R values in v for the sparse vector
        indices = v.argtopk(R).compute()
        
        data = v[indices].compute()  #Do I need to delete any of these intermediate variables? 
        #indices, data = client.compute(v.argtopk(R,axis=0), v.topk(R))
        #let's make the sparse vector.
        sv = sparse.COO(indices,data,shape=(P),sorted=False)
        sv = da.from_array(sv)
        
        # Broadcast the sparse vector.
        _V_ = client.scatter(sv,broadcast=True)

        # P1: Matrix-vector multiplication step. Computes u.
        u_new = da.matmul(S,_V_.result()) 
        print(u_new.shape)
        
        # Subtract off the mean and normalize.
        u_new -= u_new.mean()
        u_new /= sla.norm(u_new,axis = 0)
        u_new = u_new.compute()
        
        # Update for the next iteration.
        delta = sla.norm(u_old - u_new) #Should u_old be _U_?
        u_old = u_new
        num_iterations += 1
    print(f'{m}:\t{num_iterations} iterations')  
    # Save the newly-computed u and v to the output files;
    with open(file_D, "a+") as fD:
        np.savetxt(fD, u_new, fmt = "%.6f", newline = " ")
        fD.write("\n")
    with open(file_z, "a+") as fz:
        np.savetxt(fz, sv.compute().todense(), fmt = "%.6f", newline = " ")
        fz.write("\n")

    
    # P4: Deflation step. Update the primary data matrix S. 
    _U_ = client.scatter(u_new, broadcast=True)
    _V_ = client.scatter(sv, broadcast=True)

    
    #if args['debug']: print(m)
    print(m)
    
    S -= da.blockwise(operator.mul, 'ij', _U_.result(), 'i', _V_.result(), 'j', dtype='f8')
    
    S.persist()
    
    
    print(datetime.datetime.now())
    #if args['debug']: print(datetime.datetime.now())
    process = psutil.Process(os.getpid())
    print(process.memory_info().rss)

Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
0:	8 iterations
0
2020-02-15 16:32:35.514387
333496320
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
1:	4 iterations
1
2020-02-15 16:32:37.672901
334376960
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
2:	8 iterations
2
2020-02-15 16:32:42.058429
337108992
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
3:	11 iterations
3
2020-02-15 16:32:48.556386
340762624
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
4:	6 iterations
4
2020-02-15 16:32:52.571757
343293952
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(

distributed.utils - ERROR - '<' not supported between instances of 'NoneType' and 'NoneType'
Traceback (most recent call last):
  File "/Users/calebcrumley/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/Users/calebcrumley/anaconda3/lib/python3.7/site-packages/distributed/dashboard/components/scheduler.py", line 1729, in graph_doc
    graph = TaskGraph(scheduler, sizing_mode="stretch_both")
  File "/Users/calebcrumley/anaconda3/lib/python3.7/site-packages/distributed/dashboard/components/scheduler.py", line 1123, in __init__
    self.layout = GraphLayout(scheduler)
  File "/Users/calebcrumley/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/graph_layout.py", line 39, in __init__
    self.scheduler, dependencies=dependencies, priority=priority
  File "/Users/calebcrumley/anaconda3/lib/python3.7/site-packages/distributed/diagnostics/graph_layout.py", line 43, in update_graph
    stack = sorted(dependencies, key=lambda k

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
11:	11 iterations
11
2020-02-15 16:33:24.055010
379899904
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
12:	4 iterations
12
2020-02-15 16:33:26.653247
382459904
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
13:	6 iterations
13
2020-02-15 16:33:30.027731
386650112
Lets start the inner loop
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
14:	6 iterations
14
2020-02-15 16:33:33.246956
390717440


dask.array<sub, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [19]:
vector = da.random.random(50).compute()
matrix = da.random.random((50,50))
da.matmul(matrix,vector).compute()

array([15.6986661 , 13.49704217, 14.49478104, 14.34828753, 11.16741423,
       13.20684787, 14.31670847, 11.90125564, 12.25993799, 14.7672654 ,
       14.02872351, 14.69641608, 15.44056484, 14.0873929 , 13.19715546,
       12.199339  , 12.82789967, 14.69915729, 12.15378403, 13.30849708,
       13.35539721, 14.74513007, 14.47226637, 15.72075073, 16.7174969 ,
       15.00006601, 12.88702554, 12.00029295, 13.7919594 , 12.53961338,
       13.44753476, 12.72551379, 13.61387412, 12.32757044, 14.08852618,
       14.50861203, 13.42515142, 12.53537786, 15.05712115, 13.94881751,
       12.42585336, 11.19918476, 13.17296569, 11.55884996, 13.43687351,
       14.71668385, 14.38711415, 11.54125892, 13.5259135 , 14.02797357])